In [ ]:
import numpy as np
import pandas as pd
import sqlite3 as sql
import plotly.graph_objs as go ### para gráficos
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from ipywidgets import interact ## para análisis interactivo
from sklearn import neighbors ### basado en contenido un solo producto consumido
import joblib
from sklearn.neighbors import NearestNeighbors

In [ ]:
# Conexion con Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Cambio de directorio de Trabajo
import os
import sys
path = '/content/drive/MyDrive/Mod2/ANALITICA3'
os.chdir(path)
sys.path.append(path)

In [ ]:
import sqlite3 as sql
import pandas as pd
import numpy as np

conn=sql.connect('/content/drive/MyDrive/Mod2/ANALITICA3/data/db_movies')
cur=conn.cursor() ###para funciones que ejecutan sql en base de datos

In [ ]:
### para verificar las tablas que hay disponibles
cur.execute("SELECT name FROM sqlite_master where type='table' ")
cur.fetchall()

[('ratings',),
 ('movies',),
 ('usuarios_sel',),
 ('peliculas_sel',),
 ('ratings_final',),
 ('movies_final',),
 ('full_ratings',),
 ('movie_genres',),
 ('df_genres',)]

In [ ]:
# Cargas la tabla que ya contiene los datos procesados
tabla_completa = pd.read_sql_query("SELECT * FROM full_ratings", conn)

tabla_completa

,user_id,movie_id,movie_title,movie_genres,movie_rating,movie_timestamp,movie_year
0,1,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,4.0,964982703,1995
1,1,3,Grumpier Old Men,Comedy|Romance,4.0,964981247,1995
2,1,6,Heat,Action|Crime|Thriller,4.0,964982224,1995
3,1,47,Seven (a.k.a. Se7en),Mystery|Thriller,5.0,964983815,1995
4,1,50,"Usual Suspects, The",Crime|Mystery|Thriller,5.0,964982931,1995
...,...,...,...,...,...,...,...
25973,607,3785,Scary Movie,Comedy|Horror,2.0,964743188,2000
25974,607,3793,X-Men,Action|Adventure|Sci-Fi,3.0,964743155,2000
25975,607,3994,Unbreakable,Drama|Sci-Fi,3.0,997847202,2000
25976,607,3996,"Crouching Tiger, Hidden Dragon (Wo hu cang long)",Action|Drama|Romance,2.0,997847173,2000


In [ ]:
tabla_completa.info()#Se confirma que año sea de tipo entero

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25978 entries, 0 to 25977
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   user_id          25978 non-null  int64  
 1   movie_id         25978 non-null  int64  
 2   movie_title      25978 non-null  object 
 3   movie_genres     25978 non-null  object 
 4   movie_rating     25978 non-null  float64
 5   movie_timestamp  25978 non-null  int64  
 6   movie_year       25978 non-null  int64  
dtypes: float64(1), int64(4), object(2)
memory usage: 1.4+ MB


In [ ]:
#se escala el año para que este en el mismo rango
sc = StandardScaler()
tabla_completa[["year_sc"]]=sc.fit_transform(tabla_completa[['movie_year']])

Este codigo de abajo es provisional mientras averiguo como traer la que monte en el sql :)

In [ ]:
# Separar los géneros por el carácter '|' y expandirlos en múltiples filas, se hace de esta manera ya que sqlite no tiene esa funcion SALE EN FILAS
df_genres = tabla_completa.assign(movie_genres=tabla_completa['movie_genres'].str.split('|')).explode('movie_genres')

# Renombrar para claridad
df_genres = df_genres.rename(columns={'movie_genres': 'genre'})

df_genres

,user_id,movie_id,movie_title,genre,movie_rating,movie_timestamp,movie_year,year_sc
0,1,1,Toy Story,Adventure,4.0,964982703,1995,0.116708
0,1,1,Toy Story,Animation,4.0,964982703,1995,0.116708
0,1,1,Toy Story,Children,4.0,964982703,1995,0.116708
0,1,1,Toy Story,Comedy,4.0,964982703,1995,0.116708
0,1,1,Toy Story,Fantasy,4.0,964982703,1995,0.116708
...,...,...,...,...,...,...,...,...
25975,607,3994,Unbreakable,Sci-Fi,3.0,997847202,2000,0.548411
25976,607,3996,"Crouching Tiger, Hidden Dragon (Wo hu cang long)",Action,2.0,997847173,2000,0.548411
25976,607,3996,"Crouching Tiger, Hidden Dragon (Wo hu cang long)",Drama,2.0,997847173,2000,0.548411
25976,607,3996,"Crouching Tiger, Hidden Dragon (Wo hu cang long)",Romance,2.0,997847173,2000,0.548411


In [ ]:
#sacar las filas que no se van a utilizar
movie_dum1 = df_genres.drop(columns = ['user_id', 'movie_rating', 'movie_timestamp', 'movie_year', 'movie_title'])
movie_dum1

,movie_id,genre,year_sc
0,1,Adventure,0.116708
0,1,Animation,0.116708
0,1,Children,0.116708
0,1,Comedy,0.116708
0,1,Fantasy,0.116708
...,...,...,...
25975,3994,Sci-Fi,0.548411
25976,3996,Action,0.548411
25976,3996,Drama,0.548411
25976,3996,Romance,0.548411


In [ ]:
movie_dum1['genre'].nunique()# se verifica cuantos generos únicos hay

19

In [ ]:
#se dumiza los generos
col_dum = ['genre']
movie_dum2 = pd.get_dummies(movie_dum1, columns = col_dum)
movie_dum2

,movie_id,year_sc,genre_Action,genre_Adventure,genre_Animation,genre_Children,genre_Comedy,genre_Crime,genre_Documentary,genre_Drama,...,genre_Film-Noir,genre_Horror,genre_IMAX,genre_Musical,genre_Mystery,genre_Romance,genre_Sci-Fi,genre_Thriller,genre_War,genre_Western
0,1,0.116708,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
0,1,0.116708,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
0,1,0.116708,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
0,1,0.116708,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
0,1,0.116708,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25975,3994,0.548411,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
25976,3996,0.548411,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
25976,3996,0.548411,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
25976,3996,0.548411,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False


In [ ]:
#Al quedar varios registros por filas de la misma pelicula pero todas con genero diferente se agrupan para disminuir la cantidad de datos a procesar a solamente un registro de pelicula
movie_unificado = movie_dum2.groupby('movie_id', as_index=False).max()
movie_unificado

,movie_id,year_sc,genre_Action,genre_Adventure,genre_Animation,genre_Children,genre_Comedy,genre_Crime,genre_Documentary,genre_Drama,...,genre_Film-Noir,genre_Horror,genre_IMAX,genre_Musical,genre_Mystery,genre_Romance,genre_Sci-Fi,genre_Thriller,genre_War,genre_Western
0,1,0.116708,False,True,True,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,2,0.116708,False,True,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,3,0.116708,False,False,False,False,True,False,False,False,...,False,False,False,False,False,True,False,False,False,False
3,6,0.116708,True,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,True,False,False
4,7,0.116708,False,False,False,False,True,False,False,False,...,False,False,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,109374,1.757178,False,False,False,False,True,False,False,True,...,False,False,False,False,False,False,False,False,False,False
446,109487,1.757178,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,True,False,False,False
447,112852,1.757178,True,True,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
448,116797,1.757178,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,True,True,False


In [ ]:
#se define el modelo knn con 10 recomendaciones, se usa la métrica coseno
model = neighbors.NearestNeighbors(n_neighbors=10, metric='cosine')

In [ ]:
#se entrena el modelo
model.fit(movie_unificado)

NearestNeighbors(metric='cosine', n_neighbors=10)

In [ ]:
#se definen los arrays de distancia(en terminos del coseno) y los ids para saber que pelicula es
dist, idlist = model.kneighbors(movie_unificado)

In [ ]:
#distancias mas cercanas, entre mas cercano a cero mas se parecen las peliculas, esas son las que se van a recomendar
distancias = pd.DataFrame(dist)
distancias

,0,1,2,3,4,5,6,7,8,9
0,1.110223e-16,2.280088e-01,5.068225e-01,5.424720e-01,5.574985e-01,5.600825e-01,5.710613e-01,5.739846e-01,5.755691e-01,5.804225e-01
1,0.000000e+00,2.182238e-01,2.280088e-01,2.342150e-01,2.376607e-01,2.400238e-01,2.408644e-01,2.423046e-01,2.423688e-01,2.424727e-01
2,1.110223e-16,2.909482e-02,5.259151e-02,8.107575e-02,8.123920e-02,8.125420e-02,8.460363e-02,8.528064e-02,9.065122e-02,9.199318e-02
3,2.220446e-16,2.179105e-02,2.736497e-02,3.305167e-02,3.353352e-02,3.516810e-02,3.563989e-02,3.574538e-02,3.615073e-02,3.618626e-02
4,0.000000e+00,6.599910e-03,1.335883e-02,1.384902e-02,1.501230e-02,1.584226e-02,1.653148e-02,1.804603e-02,1.829202e-02,1.860390e-02
...,...,...,...,...,...,...,...,...,...,...
445,0.000000e+00,4.398715e-11,4.722633e-11,4.828848e-11,9.708967e-11,1.157922e-10,1.260567e-10,1.416716e-10,1.440433e-10,1.654562e-10
446,0.000000e+00,1.203823e-10,1.294864e-10,1.415701e-10,1.587841e-10,1.617699e-10,1.670143e-10,1.938836e-10,2.051136e-10,2.107294e-10
447,1.110223e-16,3.389744e-11,7.206102e-11,1.101033e-10,1.203823e-10,1.273806e-10,1.322848e-10,1.504058e-10,1.641960e-10,1.644032e-10
448,1.110223e-16,8.242118e-11,8.390788e-11,1.157922e-10,1.582127e-10,1.615102e-10,1.767115e-10,1.913122e-10,1.938836e-10,2.240005e-10


In [ ]:
#Para saber a que id de pelicula es
id_list = pd.DataFrame(idlist)#
id_list

,0,1,2,3,4,5,6,7,8,9
0,0,1,2,4,5,6,9,10,17,13
1,1,5,0,13,17,29,21,27,28,30
2,2,4,6,15,9,8,10,11,17,20
3,3,5,10,7,18,20,14,12,21,26
4,4,6,15,9,8,11,10,17,20,22
...,...,...,...,...,...,...,...,...,...,...
445,445,444,437,436,414,448,400,449,443,426
446,446,447,439,449,437,436,445,448,442,430
447,447,449,439,419,446,440,430,442,424,441
448,448,437,436,445,432,444,443,423,446,440


In [ ]:
#función interactiva para consultar las recomendaciones
def MovieRecommender(movie_name = list(tabla_completa['movie_title'].value_counts().index)):
    movie_list_name = []
    movie_id = tabla_completa[tabla_completa['movie_title'] == movie_name].index
    movie_id = movie_id[0]
    for newid in idlist[movie_id]:
        movie_list_name.append(tabla_completa.loc[newid].movie_title)
    return movie_list_name

In [ ]:
print(interact(MovieRecommender))

interactive(children=(Dropdown(description='movie_name', options=('Forrest Gump', 'Pulp Fiction', 'Shawshank R…

<function MovieRecommender at 0x7fb0f43af600>
